## Reproduce the result of *SmoothGrad: removing noise by adding noise*

Link to the paper: https://arxiv.org/pdf/1706.03825.pdf

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
from torch.autograd import grad,Variable

import torchvision.models as models
#------------------------------
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os,random
import attack as A
from utils import *

os.environ['CUDA_VISIBLE_DEVICES']='0'
USE_GPU=True

torch.manual_seed(123)
torch.cuda.manual_seed_all(123)
random.seed(123)

%load_ext autoreload
%autoreload 2

infer = Infer(USE_GPU)
attack = A.Attack(USE_GPU)

def print_top_pred(p):
    sorted_idx = np.argsort(p)[::-1][:5]
    print('------------->')
    for i,idx in enumerate(sorted_idx):
        print('(%.0f,%d,%s)\t'%(100*p[idx],idx,idx2class[idx]),end='')
    print('')

def load_model(model):
    net = model(pretrained=True)
    net = nn.Sequential(net,nn.Softmax())
    if USE_GPU:
        net = net.cuda()
    return net

def load_img(filepath):
    img = Image.open(filepath)
    img = PIL2tensor(img)
    return img

In [2]:
alexnet = load_model(models.alexnet)
vgg11   = load_model(models.vgg11_bn)
vgg19   = load_model(models.vgg19_bn)
resnet152  = load_model(models.resnet152)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/leiwu/.torch/models/resnet152-b121ed2d.pth
100.0%


## Record

  - For the same adversarial example generation method, different hyper-parameters, such as step size, number of step, will causes different transferability. For instance, (B=2,nstep=20) transfer, but not for (B=1,nstep=20), and (B=10,nstep=20)
  - It seems that the adversarial examples generated by large step size transfer more easily.

In [ ]:
img = load_img(
    '/home/leiwu/lab/cleverhans/examples/nips17_adversarial_competition/dataset/images/014f0024918a7b6f.png')
model  = Ensemble([vgg11,resnet50])
model2 = resnet50

ct = lambda x: 1*A.target(x,[345]) +1*A.negative_entropy(x)
img_adv1 = attack(model,ct,img,epsilon=10,nstep=50,dt=20)
p1 = infer(model,img)
p2 = infer(model,img_adv1)
p3 = infer(model2,img)
p4 = infer(model2,img_adv1)

plt.figure(figsize=(14,4))
plt.subplot(1,2,1); imshow_th(img,idx2class[p1.argmax()])
plt.subplot(1,2,2); imshow_th(img_adv1,idx2class[p2.argmax()])

print_top_pred(p1)
print_top_pred(p2)
print_top_pred(p3)
print_top_pred(p4)

In [ ]:
img2 = torch.rand(3,224,224) + img_adv1 - img
img2 = torch.clamp(img2,0,1)
p = infer(model,img2)
print_top_pred(p)

In [ ]:
torch.rand(3,224,224).shape

In [ ]:
# p3 = infer(model2,img)
p4 = infer(model2,img_adv1)

plt.figure(figsize=(10,5))
plt.subplot(2,2,1); imshow_th(img,idx2class[p1.argmax()])
plt.subplot(2,2,2); imshow_th(img_adv1,idx2class[p2.argmax()])
plt.subplot(2,2,3);  imshow_th(img,idx2class[p3.argmax()])
plt.subplot(2,2,4);  imshow_th(img_adv1,idx2class[p4.argmax()])

In [ ]:
np.argsort(p1)[::-1][::10]

In [ ]:
idx2class[716]

## How the prediction changes along the epsilon

In [ ]:
img = Image.open('./images/doberman.png')
img = PIL2tensor(img)

ct = confidence
img_adv1 = FastGradAttack(model,ct,img,2)
img_adv2 = FastGradAttack(model,ct,img,6)
img_adv3 = FastGradAttack(model,ct,img,15)
img_adv4 = FastGradAttack(model,ct,img,20)
img_adv5 = FastGradAttack(model,ct,img,25)
p0 = infer(model,img)
p1 = infer(model,img_adv1)
p2 = infer(model,img_adv2)
p3 = infer(model,img_adv3)
p4 = infer(model,img_adv4)
p5 = infer(model,img_adv5)

plt.figure(figsize=(12,6))
plt.subplot(2,3,1); imshow(img,classes[p0.argmax()])
plt.subplot(2,3,2); imshow(img_adv1,classes[p1.argmax()])
plt.subplot(2,3,3); imshow(img_adv2,classes[p2.argmax()])
plt.subplot(2,3,4); imshow(img_adv3,classes[p3.argmax()])
plt.subplot(2,3,5); imshow(img_adv4,classes[p4.argmax()])
plt.subplot(2,3,6); imshow(img_adv5,classes[p5.argmax()])

In [ ]:
P=[torch.rand(3,4) for _ in range(5)]

In [ ]:
sum(P)

## Caveat
- Must use *eval model*, the forward propogation would change the batch statistics in *train model*

In [ ]:
test_image(model,img,smooth=True,nsamples=50,stddev=0.2)

In [ ]:
img = Image.open('./images/5.png')
img = PIL2tensor(img)
test_image(model,img,smooth=True,nsamples=10,stddev=0.2)

In [ ]:
img = Image.open('./images/16.png')
img = PIL2tensor(img)
test_image(model,img,smooth=True,nsamples=100,stddev=0.2)